In [211]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
pd.set_option("display.max_columns", 100)
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.metrics import roc_auc_score

In [212]:
df = pd.read_csv('data/kddcup.data', index_col=False, header=None)

In [213]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [214]:
df = pd.get_dummies(df, columns=[1, 2, 3]).drop(0, axis=1)

In [215]:
df.head()

,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,1_icmp,1_tcp,1_udp,2_IRC,2_X11,2_Z39_50,2_aol,2_auth,2_bgp,2_courier,2_csnet_ns,2_ctf,...,2_kshell,2_ldap,2_link,2_login,2_mtp,2_name,2_netbios_dgm,2_netbios_ns,2_netbios_ssn,2_netstat,2_nnsp,2_nntp,2_ntp_u,2_other,2_pm_dump,2_pop_2,2_pop_3,2_printer,2_private,2_red_i,2_remote_job,2_rje,2_shell,2_smtp,2_sql_net,2_ssh,2_sunrpc,2_supdup,2_systat,2_telnet,2_tftp_u,2_tim_i,2_time,2_urh_i,2_urp_i,2_uucp,2_uucp_path,2_vmnet,2_whois,3_OTH,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [216]:
X_df = df.drop(41, axis=1)
y_df = df[41]

In [217]:
X = X_df.values
y = y_df.values

In [218]:
df_smp = df.sample(n=100000)

In [219]:
len(df_smp)

100000

In [220]:
X_smp = df_smp.drop(41, axis=1).values
y_smp = df_smp.loc[:, 41].values

In [221]:
y_smp = np.array([ 1 if yi != "normal." else 0 for yi in y_smp])

In [230]:
sample_index = np.random.choice(np.arange(len(X_smp)), 10)

In [231]:
X_subsmp = X_smp[sample_index]
y_subsmp = y_smp[sample_index]

In [232]:
rf = RandomForestClassifier()

In [233]:
rf.fit(X_subsmp, y_subsmp)

RandomForestClassifier()

In [234]:
predict = rf.predict(X_smp)

In [235]:
roc_auc = roc_auc_score(predict, y_smp)

In [236]:
print(f"0回目 {roc_auc*100.0:.05}%")

0回目 96.309%


In [237]:
total_labeled = 10
for ite in range(3):
    proba = rf.predict_proba(X_smp)[:, 1]
    mlp = []
    llp = []
    mln = []
    lln = []
    for i, (x, pi, probi, y) in enumerate(zip(X_smp, predict, proba, y_smp)):
        if pi == 1 and probi >= 0.9:
            mlp += [i]
        elif pi == 1 and probi < 0.9:
            llp += [i]
        elif pi == 0 and probi <= 0.1:
            mln += [i]
        elif pi == 0 and probi > 0.1:
            lln += [i]
    new_X = X_smp[mlp]
    new_y = [1] * len(mlp)

    new_X = np.concatenate([new_X, X_smp[mln]], 0)
    new_y = np.concatenate([new_y, [0]*len(mln)], 0)

    # human check
    tmp_anomaly = []
    tmp_nominal = []
    for i in llp[:100]:
        if y_smp[i] == 1:
            tmp_anomaly += [i]
        else:
            tmp_nominal += [i]
        total_labeled += 1

    new_X = np.concatenate([new_X, X_smp[tmp_anomaly]], 0)
    new_y = np.concatenate([new_y, [1]*len(tmp_anomaly)], 0)

    new_X = np.concatenate([new_X, X_smp[tmp_nominal]], 0)
    new_y = np.concatenate([new_y, [0]*len(tmp_nominal)], 0)

    # human checkl
    tmp_anomaly = []
    tmp_nominal = []
    for i in lln[:100]:
        if y_smp[i] == 1:
            tmp_anomaly += [i]
        else:
            tmp_nominal += [i]
    total_labeled += 1

    new_X = np.concatenate([new_X, X_smp[tmp_anomaly]], 0)
    new_y = np.concatenate([new_y, [1]*len(tmp_anomaly)], 0)

    new_X = np.concatenate([new_X, X_smp[tmp_nominal]], 0)
    new_y = np.concatenate([new_y, [0]*len(tmp_nominal)], 0)

    rf = RandomForestClassifier()
    rf.fit(new_X, new_y)
    predict = rf.predict(X_smp)
    roc_auc = roc_auc_score(predict, y_smp)

    print(f"{ite+1}回目 {roc_auc*100:.05}%, ラベリングしたデータ数 {total_labeled}")

1回目 99.699%, ラベリングしたデータ数 111
2回目 99.796%, ラベリングしたデータ数 212
3回目 99.873%, ラベリングしたデータ数 313
